![NASA](http://www.nasa.gov/sites/all/themes/custom/nasatwo/images/nasa-logo.svg)
![DEVELOP](../../DEVELOP_logo.png)

---

# GDAL

### Goddard Space Flight Center

#### March 13, 2017

---

---

# What is GDAL?

---

* __Geospatial Data Abstraction Library__ - A programmatic way to do geoprocessing, providing similar capabilities to those available in ArcGIS
* Useful for building automated systems, performing repetitive analysis, etc.
* Developed by Frank Warmerdam, Sean Gilles, et al.
* Both ArcGIS and QGIS use GDAL "under the hood"
* When people say GDAL, they really mean the GDAL sub-library of the OSGeo (Open Source Geospatial Foundation) package. OSGeo has two sub-libraries:
    * gdal - reading, processing, writing raster data
    * ogr - reading, processing, writing vector data
    * This lecture will only cover gdal, but note that you will need ogr to do anything significant with vector data
* GDAL has been built to work with several different languages, including Python, R, C, and C++, but some of its most useful capabilities exist as standalone command line tools - essentially, these are their own little programs that can only be started through the command line
    * We will look at both the GDAL Python functions and some of the most useful command line tools
* [OSGeo site](http://osgeo.org)
* [GDAL Python reference](http://gdal.org/python/)
* [GDAL Python tips](https://trac.osgeo.org/gdal/wiki/PythonGotchas)

# GDAL vs. ArcPy - Pros and Cons

---

* GDAL is faster and more accurate
* GDAL typically provides more flexibility and control
* Some things can be done in GDAL that simply aren't possible (to my knowledge) in ArcGIS, such as reprojecting between certain coordinate systems
* GDAL is free and open source
* However, GDAL is significantly less user-friendly, because:
    * There is no GUI version of the software, like ArcMap for ArcGIS
    * The documentation is primarily written for programmers, and is generally light on explanation
* To summarize, you can expect a steep learning curve with GDAL, but you will ultimately be able to do some cool stuff

# Example Project: More fun with MODIS

---

To explore GDAL and allow comparisons between GDAL and ArcPy, we will examine some of the same steps we took in our project last week, namely:
* Extracting a single subdataset from a MODIS HDF file
* Opening this data with Python and doing some math
* Writing the modified data back to disk

We will also perform a new step, which is not possible with ArcGIS: reprojecting the data from its native sinusoidal coordinate system to WGS 1984.

__Important:__ Before beginning this lecture, you should have followed the GDAL installation instructions, available from the root directory of the DEVELOP Python GitHub.

### Download data

---

Download this MODIS image:  
ftp://ladsweb.nascom.nasa.gov/allData/6/MOD11A2/2016/201/MOD11A2.A2016201.h11v05.006.2016242234243.hdf 

Copy it to the directory where you want to store your data. You may already have this image from last week's ArcPy lecture.

### Get metadata

---

We want to extract a subdataset from this HDF file, but suppose we don't know what that subdataset is called? Navigate to your working directory, and type the following in the command line:

> gdalinfo MOD11A2.A2016201.h11v05.006.2016242234243.hdf

```> gdalinfo MOD11A2.A2016201.h11v05.006.2016242234243.hdf```

This is a lot of information, but it should look familiar from the HDF/netCDF lecture. If you look closely, you will seen the names and descriptions of each subdataset. For example, the full name of the daytime LST subdataset is ```HDF4_EOS:EOS_GRID:"MOD11A2.A2016201.h11v05.006.2016242234243.hdf":MODIS_Grid_8Day_1km_LST:LST_Day_1km```

### Extract subdataset

---

To extract the daytime LST subdataset, we will use one of the most useful GDAL command line tools: ```gdal_translate```. This tool is most commonly used for converting data from one file format to another. If you don't have experience running command line tools, the general format is:
* Name of tool first
* Optional parameters specified as ```-OptParam ArgumentForOptParam```
* Required arguments

In the case of ```gdal_translate```, there are two required arguments: the names of the input and output files. There are also numerous optional arguments that can be used to pack in additional functionality, such as altering the output bit depth and resampling the data. To extract the subdataset, go to your command line and type:

```> gdal_translate HDF4_EOS:EOS_GRID:"MOD11A2.A2016201.h11v05.006.2016242234243.hdf":MODIS_Grid_8Day_1km_LST:LST_Day_1km MOD11A2.A2016201.h11v05.006.2016242234243_dayLST.tif```

In this case, we are not using any optional arguments, merely providing the name of the subdataset we care about and an output file name. Now, even with copy/pasting, tab-filling, etc., it would be tedious to do this for numerous images. Let's take a look at how we might call this command from Python, using the ```subprocess``` module.

In [ ]:
import subprocess
import os

#Change system to working directory
workdir = "C:\\Users\\abhubba1\\Documents\\Python Scripts\\DEVELOP_class"
os.chdir(workdir)

MODIS_file = "MOD11A2.A2016201.h11v05.006.2016242234243.hdf"

#Send gdal_translate command to system shell, capture result, and print it
dayLST_fname = MODIS_file.rstrip('.hdf')+'_GDAL_dayLST.tif'
trans_day_cmd = ['gdal_translate', 'HDF4_EOS:EOS_GRID:"'+MODIS_file+\
                 '":MODIS_Grid_8Day_1km_LST:LST_Day_1km', dayLST_fname]
p_trans_day = subprocess.Popen(trans_day_cmd, stdout=subprocess.PIPE, 
                               stderr=subprocess.PIPE)
print(p_trans_day.communicate())

The ```subprocess.Popen``` takes a list of string arguments that comprise your command. Every separate element of a command should be its own element in this list. Note that this includes the name of an optional parameter and the argument for that parameter.

Optionally, ```subprocess.Popen``` also allows us to capture the ```stdout``` and ```stderr``` streams. These are the output and error streams for programs run in the command line, and contain output messages and errors, respectively. It is common to save these to a text file to view later, but in this case we are just printing them using the ```communicate()``` method.

### Opening raster data in Python

---

We have just covered an example of using GDAL command line tools, and calling those tools from Python. As mentioned above, many of GDAL's most useful capabilities are command line tools such as this, but GDAL also has a Python-specific library that allows us to perform some operations entirely in Python, without sending any commands to the system. In some cases, this Python library duplicates the capabilities of the command line tools, and in that case, it is generally better to use the command line tool, as they tend to be better supported. However, the Python library allows us to do some uniquely cool things, such as opening a raster as a NumPy array.

In [ ]:
import os

from osgeo import gdal
import numpy as np

gdal.UseExceptions()

#Change system to working directory
workdir = "C:\\Users\\abhubba1\\Documents\\Python Scripts\\DEVELOP_class"
os.chdir(workdir)

MODIS_file = "MOD11A2.A2016201.h11v05.006.2016242234243.hdf"
dayLST_fname = MODIS_file.rstrip('.hdf')+'_GDAL_dayLST.tif'

#Register driver for this file type
driver = gdal.GetDriverByName("GTiff")
driver.Register()
#Open raster as GDAL dataset
dayLST_dataset = gdal.Open(dayLST_fname)
#Get geotransform and projection from GDAL dataset. These contain 
#the geospatial information of the data, and we will need them 
#later to write the array back to a raster file.
geotrans = dayLST_dataset.GetGeoTransform()
proj = dayLST_dataset.GetProjection()
#Open the only band in the dataset. Note that band numbering 
#starts from 1 as far as GDAL is concerned.
dayLST_band = dayLST_dataset.GetRasterBand(1)
#Pull data from band into a NumPy array
dayLST_array = dayLST_band.ReadAsArray()
#Get the NoData value for this band
fillval = dayLST_band.GetNoDataValue()
#Create a new masked array, where all areas of NoData are masked 
#out
dayLST_ma_array = np.ma.masked_equal(dayLST_array, fillval)
#Empty band and dataset objects to avoid lock issues later. Be 
#sure to empty the band object first, as there can be problems 
#otherwise.
dayLST_band = None
dayLST_dataset = None

There's a lot going on here, so let's unpack it:
```Python
#Register driver for this file type
driver = gdal.GetDriverByName("GTiff")
driver.Register()
```
GDAL has a separate driver for each supported raster format, and it is necessary to register this driver so that GDAL knows how to open a given file. It is also possible to register all drivers at once, but we will need our driver later to write the data to disk, so we want to use ```GetDriverByName```.
```Python
#Open raster as GDAL dataset
dayLST_dataset = gdal.Open(dayLST_fname)
#Get geotransform and projection from GDAL dataset. These contain 
#the geospatial information of the data, and we will need them 
#later to write the array back to a raster file.
geotrans = dayLST_dataset.GetGeoTransform()
proj = dayLST_dataset.GetProjection()
```
This first line opens the raster as a GDAL dataset object, which you can think of as the umbrella which encompasses all aspects of a raster, including both metadata and data. After that, we need to retrieve two pieces of metadata, the geotransform and the projection. Opening raster data in a NumPy array is powerful, but NumPy can't store the geospatial information, so we need to keep track of that ourselves.
```Python
#Open the only band in the dataset. Note that band numbering 
#starts from 1 as far as GDAL is concerned.
dayLST_band = dayLST_dataset.GetRasterBand(1)
#Pull data from band into a NumPy array
dayLST_array = dayLST_band.ReadAsArray()
```
The dataset object contains all of the bands in the dataset, which in this case is only one band. We are retrieving that band and using ```ReadAsArray``` to load it into a NumPy array.
```Python
#Get the NoData value for this band
fillval = dayLST_band.GetNoDataValue()
#Create a new masked array, where all areas of NoData are masked 
#out
dayLST_ma_array = np.ma.masked_equal(dayLST_array, fillval)
```
Another important piece of metadata is the NoData value. As with the geotransform and projection, we need to keep track of this for writing the data later. However, we also need it now, because we are going to create a NumPy masked array.

A masked array is the same as a normal NumPy array, except it contains a second Boolean array that is used for masking. Any cell that is ```True``` in this Boolean array will be masked in the data, and these masked cells will not be included in any NumPy operations. In this case, we are using ```masked_equal``` to create an array that is masked everywhere the NoData value is present. Now we can do math with this array and not worry about NoData values throwing things off.
```Python
#Empty band and dataset objects to avoid lock issues later. Be 
#sure to empty the band object first, as there can be problems 
#otherwise.
dayLST_band = None
dayLST_dataset = None
```
As was the case when working with ArcPy, leaving the GDAL objects lying around can lead to issues later, as these objects point to the files on disk.

### Raster math

---

Or, more properly, NumPy math. While opening this data as a NumPy array was, let's be honest, a pain in the butt, now we have a NumPy array of our data, and we can do just about anything we want with it. NumPy has an expansive library of mathematical operations - it is called Numerical Python for a reason. For even more possibilities, check out the SciPy library, which is built on top of NumPy.

In the interests of time and continuity with the ArcPy lecture, we will simply rescale the data. As with ArcPy objects, NumPy will automatically apply simple mathematical operations elementwise to the array.

In [ ]:
scale = 0.02
dayLST_array_sc = dayLST_ma_array * scale

### Writing the array back to a raster

---

After we have manipulated the data to our heart's content, we need to write it back to a raster format. In many ways, this is just the reverse of what we did above to read it into an array.

In [ ]:
gdal.UseExceptions()

#Create new dataset to contain output
scale_fname = MODIS_file.rstrip('.hdf')+'_GDAL_scale.tif'
out_dataset = driver.Create(scale_fname, dayLST_array_sc.shape[1], 
                            dayLST_array_sc.shape[0], eType = gdal.GDT_UInt16)
#Set geotransform and projection of output dataset
out_dataset.SetGeoTransform(geotrans)
out_dataset.SetProjection(proj)
#Create a band for our data
out_band = out_dataset.GetRasterBand(1)
#Write our data to the band
out_band.WriteArray(dayLST_array_sc)
#Tell the raster which value signifies NoData
out_band.SetNoDataValue(fillval)
#Write the data from memory to disk. Not strictly necessary, as 
#this should occur anyway at some point, but it is good practice.
out_band.FlushCache()
#Clear band and dataset to avoid lock problems
out_band = None
out_dataset = None

Let's unpack this one as well:
```Python
#Create new dataset to contain output
scale_fname = MODIS_file.rstrip('.hdf')+'_GDAL_scale.tif'
out_dataset = driver.Create(scale_fname, dayLST_array_sc.shape[1], 
                            dayLST_array_sc.shape[0], eType = gdal.GDT_UInt16)
```
Here we use our driver object to create a new dataset with the file name, dimensions, and data type of our choosing. Note the data type has to be specified as a ```GDALDataType``` object, the reference for which can be found [here](http://www.gdal.org/gdal_8h.html#a22e22ce0a55036a96f652765793fb7a4).
```Python
#Set geotransform and projection of output dataset
out_dataset.SetGeoTransform(geotrans)
out_dataset.SetProjection(proj)
```
As mentioned above, the NumPy array isn't storing our geospatial metadata, so here we give that information to our new dataset object.
```Python
#Create a band for our data
out_band = out_dataset.GetRasterBand(1)
#Write our data to the band
out_band.WriteArray(dayLST_array_sc)
```
These lines create a band for the output data, and assign the data to that band. We could create more bands in the same way if we were working with multiband rasters.
```Python
#Tell the raster which value signifies NoData
out_band.SetNoDataValue(fillval)
```
A NoData value is just a number unless the raster knows which number signifies NoData. This line provides that information.
```Python
#Write the data from memory to disk. Not strictly necessary, as 
#this should occur anyway at some point, but it is good practice.
out_band.FlushCache()
```
Even though we "wrote" the array above, this is line that actually writes the data to disk. This should happen anyway when we "close" the dataset below, but it is good practice.
```Python
#Clear band and dataset to avoid lock problems
out_band = None
out_dataset = None
```
Clearing out these objects is helpful to avoid lock problems, as mentioned above. You can think of this as "closing" the file, from GDAL's perspective.

### Reprojecting raster data

---

Up until this point, we have been duplicating the capabilities of ArcGIS in GDAL. This can be handy because GDAL generally performs things more quickly, more accurately, and provides more control over the process. It is also free, which is nice if you don't have an ArcGIS license. However, you have probably noticed that doing these things in GDAL is, on the surface at least, more complex than doing them with ArcPy. We haven't covered anything where you strictly _need_ GDAL. So, in this last section, we will go over something that strictly requires GDAL: reprojecting data from MODIS sinusoidal to WGS 1984.

To do this, we will be using the extremely powerful ```gdalwarp``` command line tool. In addition to reprojecting, this tool can be used for clipping, masking, resampling, etc., but it is designed around reprojection. To do this from the command line, type:
```
> gdalwarp -t_srs "EPSG:4326" in_file out_file
```
Replace ```in_file``` with the name of our MODIS raster, and replace ```out_file``` with an output file name of your choosing. To call this tool from Python, you would use ```subprocess```, as above:

In [ ]:
import subprocess
import os

#Change system to working directory
workdir = "C:\\Users\\abhubba1\\Documents\\Python Scripts\\DEVELOP_class"
os.chdir(workdir)

MODIS_file = "MOD11A2.A2016201.h11v05.006.2016242234243.hdf"
scale_fname = MODIS_file.rstrip('.hdf')+'_GDAL_scale.tif'

#Send gdalwarp command to system shell, capture result, and print it
reproj_fname = MODIS_file.rstrip('.hdf')+'_GDAL_reproj.tif'
reproj_cmd = ['gdalwarp', '-t_srs', 'EPSG:4326', scale_fname, reproj_fname]
p_reproj = subprocess.Popen(reproj_cmd, stdout=subprocess.PIPE, 
                            stderr=subprocess.PIPE)
print(p_reproj.communicate())